In [3]:

import numpy as np
import pandas as pd
import tqdm

from datetime import date, datetime
import math
from tqdm import tqdm
import time

In [4]:

df = pd.read_csv("/opt/ml/before/input/data/train_data.csv")
df = df.sort_values(by=["userID", "Timestamp"]).reset_index(drop=True)

In [ ]:
train = pd.read_csv("/opt/ml/before/input/data/train_data.csv")
test = pd.read_csv('/opt/ml/before/input/data/test_data.csv')

test['answerCode'] = np.full((test.shape[0]), 0)

train['train_test'] = train['answerCode']
test['train_test'] = np.full((test.shape[0]), -1)

df = pd.concat([train, test])
df = df.sort_values(by=["userID", "Timestamp"]).reset_index(drop=True)

array([0, 1])

In [ ]:
df["user_correct_answer"] = (
    df.groupby("userID")["answerCode"]
    .transform(lambda x: x.cumsum().shift(1))
    .fillna(0)
)
df["user_total_answer"] = df.groupby("userID")["answerCode"].cumcount() + 1
df["user_acc"] = (df["user_correct_answer"] / df["user_total_answer"])
df['answerCode'] = df['train_test']
df.tail(20)

In [5]:
def fe(df):
    
    # assessmentItemID split
    df['assessmentItemID0'] = df['assessmentItemID'].str[2]
    df['assessmentItemID1'] = df['assessmentItemID'].str[4:7]
    df['assessmentItemID2'] = df["assessmentItemID"].apply(lambda x: int(x[-2:]))
    
    # testID split
    df['testId0'] = df['assessmentItemID0']
    df['testId1'] = df['assessmentItemID1']
    
    # assessmentItemid join
    df['as0_as1'] =  np.array([str(i)+'_'+str(j) for i, j in zip(list(df.assessmentItemID0), list(df.assessmentItemID1))])
    df['as0_as2'] =  np.array([str(i)+'_'+str(j) for i, j in zip(list(df.assessmentItemID0), list(df.assessmentItemID2))])
    df['as1_as2'] =  np.array([str(i)+'_'+str(j) for i, j in zip(list(df.assessmentItemID1), list(df.assessmentItemID2))])
    
    # time split
    df['month'] = df.Timestamp.str[5:7]
    df['hour'] = df.Timestamp.str[11:13]

    yy = df.Timestamp.str[:4]
    mm = df.Timestamp.str[5:7]
    dd = df.Timestamp.str[8:10]

    y = [int(y) for y in list(yy)]
    m = [int(m) for m in list(mm)]
    d = [int(d) for d in list(dd)]

    df['week'] = [date(i,j,k).weekday() for i,j,k in zip(y,m,d)]
    w_h = [str(i)+'_'+j for i, j in zip(list(df.week), list(df.hour))]
    df['week_hour'] = w_h
    df['day'] = dd
    df['minute'] = df.Timestamp.str[14:16]
    df['second'] = df.Timestamp.str[17:19]


    # elapsed time
    def convert_time(s):
        timestamp = time.mktime(
            datetime.strptime(s, "%Y-%m-%d %H:%M:%S").timetuple()
        )
        return int(timestamp)

    df['Time'] = df.Timestamp.apply(convert_time)
    
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    diff = df.loc[:, ['userID', 'Timestamp']].groupby('userID').diff().fillna(pd.Timedelta(seconds=0))
    diff = diff.fillna(pd.Timedelta(seconds=0))
    diff = diff['Timestamp'].apply(lambda x: x.total_seconds())

    df['elapsed'] = diff
    df["elapsed_log"] = df["elapsed"].apply(lambda x: math.log(x + 1))
    
    df["elapsed"] = df["elapsed"].apply(
        lambda x: x if x < 1800 and x >= 0 else 0)

    def elap(x):
        if x == 0:
            return 0
        elif x < 9:
            return 1
        elif x < 301:
            return 2
        elif x < 601:
            return 3
        elif x < 1201:
            return 4
        elif x < 1501:
            return 5
        elif x < 1801:
            return 6
        else:
            return 7

    df['elapsed_cate'] = df.elapsed.apply(lambda x : elap(x))
    
    correct_k = df.groupby(["KnowledgeTag"])["answerCode"].agg(["mean", "std"])
    correct_k.columns = ["tag_mean", "tag_std"]

    correct_a = df.groupby(["assessmentItemID0"])["answerCode"].agg(["mean", "std"])
    correct_a.columns = ["ass0_mean", "ass0_std"]
    correct_a1 = df.groupby(["assessmentItemID1"])["answerCode"].agg(["mean", "std"])
    correct_a1.columns = ["ass1_mean", "ass1_std"]
    correct_a2 = df.groupby(["assessmentItemID2"])["answerCode"].agg(["mean", "std"])
    correct_a2.columns = ["ass2_mean", "ass2_std"]

    
    df = pd.merge(df, correct_k, on=["KnowledgeTag"], how="left")
    df = pd.merge(df, correct_a, on=["assessmentItemID0"], how="left")
    df = pd.merge(df, correct_a1, on=["assessmentItemID1"], how="left")
    df = pd.merge(df, correct_a2, on=["assessmentItemID2"], how="left")
    
    
    df['test0_mean'] = df['ass0_mean']
    df['test0_std'] = df['ass0_std']
    df['test1_mean'] = df['ass1_mean']
    df['test1_std'] = df['ass1_std']
    
    return df

In [9]:
# elo probability

def ELO_function(df):
    def get_new_theta(is_good_answer, beta, left_asymptote, theta, nb_previous_answers):
        return theta + learning_rate_theta(nb_previous_answers) * (
            is_good_answer - probability_of_good_answer(theta, beta, left_asymptote)
        )

    def get_new_beta(is_good_answer, beta, left_asymptote, theta, nb_previous_answers):
        return beta - learning_rate_beta(nb_previous_answers) * (
            is_good_answer - probability_of_good_answer(theta, beta, left_asymptote)
        )

    def learning_rate_theta(nb_answers):
        return max(0.3 / (1 + 0.01 * nb_answers), 0.04)

    def learning_rate_beta(nb_answers):
        return 1 / (1 + 0.05 * nb_answers)

    def probability_of_good_answer(theta, beta, left_asymptote):
        return left_asymptote + (1 - left_asymptote) * sigmoid(theta - beta)

    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    def estimate_parameters(answers_df, granularity_feature_name="assessmentItemID"):
        item_parameters = {
            granularity_feature_value: {"beta": 0, "nb_answers": 0}
            for granularity_feature_value in np.unique(
                answers_df[granularity_feature_name]
            )
        }
        student_parameters = {
            student_id: {"theta": 0, "nb_answers": 0}
            for student_id in np.unique(answers_df.userID)
        }

        print("Parameter estimation is starting...")

        for student_id, item_id, left_asymptote, answered_correctly in tqdm(
            zip(
                answers_df.userID.values,
                answers_df[granularity_feature_name].values,
                answers_df.left_asymptote.values,
                answers_df.answerCode.values,
            )
        ):
            theta = student_parameters[student_id]["theta"]
            beta = item_parameters[item_id]["beta"]

            item_parameters[item_id]["beta"] = get_new_beta(
                answered_correctly,
                beta,
                left_asymptote,
                theta,
                item_parameters[item_id]["nb_answers"],
            )
            student_parameters[student_id]["theta"] = get_new_theta(
                answered_correctly,
                beta,
                left_asymptote,
                theta,
                student_parameters[student_id]["nb_answers"],
            )

            item_parameters[item_id]["nb_answers"] += 1
            student_parameters[student_id]["nb_answers"] += 1

        print(f"Theta & beta estimations on {granularity_feature_name} are completed.")
        return student_parameters, item_parameters

    def gou_func(theta, beta):
        return 1 / (1 + np.exp(-(theta - beta)))

    df["left_asymptote"] = 0

    print(f"Dataset of shape {df.shape}")
    print(f"Columns are {list(df.columns)}")

    student_parameters, item_parameters = estimate_parameters(df)

    prob = [
        gou_func(student_parameters[student]["theta"], item_parameters[item]["beta"])
        for student, item in zip(df.userID.values, df.assessmentItemID.values)
    ]

    df["elo_prob"] = prob

    return df

In [ ]:
# label encoding

cate = ['KnowledgeTag', 'month', 'hour', 'week', 'elapsed_cate', 'testId0', 'testId1',
        'assessmentItemID0', 'assessmentItemID1', 'assessmentItemID2', 'day', 'minute', 'second',
        'as0_as1', 'as0_as2', 'as1_as2', 'assessmentItemID', 'week_hour']

conti = ['elapsed' , 'tag_mean', 'tag_std', 'ass0_mean', 'ass0_std', 'elapsed_log',
        'ass1_mean', 'ass1_std', 'ass2_mean', 'ass2_std', 'user_correct_answer', 'Time'
        'user_total_answer', 'user_acc', 'recAccuracy', 'recCount', 'solve_order']

for i in cate:
    cate2label = {j:i for i,j in enumerate(df[i].unique())}
    df[i] = df[i].map(cate2label)

In [ ]:
# train & test splilt

train = df[df['answerCode'] != -1]
test = df[df['answerCode'] == -1]

train.to_csv('../../elo.csv', index=False)
test.to_csv('../../infer.csv', index=False)

#### Inference file

In [ ]:
infer_file = df[df['rr'] == -1]
infer_file.drop(['r', 'rr'], axis=1)
infer_file.to_csv('../../../data/infer.csv', index=False)
df = df[df['rr'] != -1]
df = df.drop(['r', 'rr'], axis=1)
df.to_csv('../../../data/elo.csv', index=False)

In [14]:
a = pd.read_csv('/opt/ml/input/data/test_elo.csv')

elo = []
for i in range(len(a)):
    if a['answerCode'].iloc[i] == -1:
        elo.append(a['elo_prob'].iloc[i-1])
    else:
        elo.append(a['elo_prob'].iloc[i])
        
a['elo_prob'] = elo

a.to_csv('/opt/ml/input/data/test_elo.csv', index=False)

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,assessmentItemID0,assessmentItemID1,assessmentItemID2,testId0,...,ass1_mean,ass1_std,ass2_mean,ass2_std,test0_mean,test0_std,test1_mean,test1_std,left_asymptote,elo_prob
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,6,001,1,6,...,0.710348,0.453616,0.749916,0.433062,0.709232,0.454118,0.710348,0.453616,0,0.979350
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,6,001,2,6,...,0.710348,0.453616,0.720062,0.448969,0.709232,0.454118,0.710348,0.453616,0,0.970579
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,6,001,3,6,...,0.710348,0.453616,0.687773,0.463402,0.709232,0.454118,0.710348,0.453616,0,0.942168
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,6,001,4,6,...,0.710348,0.453616,0.663364,0.472560,0.709232,0.454118,0.710348,0.453616,0,0.972448
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,6,001,5,6,...,0.710348,0.453616,0.599134,0.490075,0.709232,0.454118,0.710348,0.453616,0,0.957230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,3,071,5,3,...,0.638852,0.480351,0.599134,0.490075,0.702238,0.457275,0.638852,0.480351,0,0.281925
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,4,165,1,4,...,0.710708,0.453464,0.749916,0.433062,0.684056,0.464891,0.710708,0.453464,0,0.632742
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,4,165,2,4,...,0.710708,0.453464,0.720062,0.448969,0.684056,0.464891,0.710708,0.453464,0,0.615929
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,4,165,3,4,...,0.710708,0.453464,0.687773,0.463402,0.684056,0.464891,0.710708,0.453464,0,0.738609


In [ ]:
def add_last_problem(df):
    new = []
    pre = df["testId"][0]
    for idx in df["testId"]:
        if pre != idx:
            new[-1] = -1
            pre = idx
        new.append(0)
    df["last_problem"] = new
    return df

dtype = {"userID": "int16", "answerCode": "int8", "KnowledgeTag": "int16"}

df = pd.read_csv('../../data/test_data.csv', dtype=dtype, parse_dates=["Timestamp"])
df = df.sort_values(by=["userID", "Timestamp"]).reset_index(drop=True)

df["problem_number"] = df["assessmentItemID"].apply(lambda x: int(x[-3:]))

correct_t = df.groupby(["testId"])["answerCode"].agg(["mean", "sum"])
correct_t.columns = ["test_mean", "test_sum"]
correct_k = df.groupby(["KnowledgeTag"])["answerCode"].agg(["mean", "sum"])
correct_k.columns = ["tag_mean", "tag_sum"]
correct_a = df.groupby(["assessmentItemID"])["answerCode"].agg(["mean", "sum"])
correct_a.columns = ["ass_mean", "ass_sum"]
correct_p = df.groupby(["problem_number"])["answerCode"].agg(["mean", "sum"])
correct_p.columns = ["prb_mean", "prb_sum"]

In [ ]:
def feature_engineering(df):
    print("-" * 20, "Feature Engineering Start", "-" * 20)
    start_time = time.time()
    # 유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
    df.sort_values(by=["userID", "Timestamp"], inplace=True)
    df = add_last_problem(df)
    
    # elo 추가
    # df = ELO_function(df)

    df["hour"] = df["Timestamp"].dt.hour
    df["dow"] = df["Timestamp"].dt.dayofweek

    # 푸는 시간
    diff = (
        df.loc[:, ["userID", "Timestamp"]]
        .groupby("userID")
        .diff()
        .fillna(pd.Timedelta(seconds=0))
    )
    diff = diff.fillna(pd.Timedelta(seconds=0))
    diff = diff["Timestamp"].apply(lambda x: x.total_seconds())
    df["elapsed"] = diff
    df["elapsed"] = df["elapsed"].apply(lambda x: x if x < 600 and x >= 0 else 0)
    def elap(x):
        if x == 0:
            return 0
        elif x < 9:
            return 1
        elif x < 101:
            return 2
        elif x < 201:
            return 3
        elif x < 301:
            return 4
        elif x < 401:
            return 5
        elif x < 501:
            return 6
        elif x < 601:
            return 7
        
    df.elapsed = df.elapsed.apply(lambda x : elap(x))

    df["grade"] = df["testId"].apply(lambda x: int(x[2]))
    df["mid"] = df["testId"].apply(lambda x: x[7:10])
    df["problem_number"] = df["assessmentItemID"].apply(lambda x: int(x[-3:]))
    df['assessmentItemID0'] = df['assessmentItemID'].str[2]
    df['assessmentItemID1'] = df['assessmentItemID'].str[4:7]
    
    correct_h = df.groupby(["hour"])["answerCode"].agg(["mean", "sum"])
    correct_h.columns = ["hour_mean", "hour_sum"]
    correct_d = df.groupby(["dow"])["answerCode"].agg(["mean", "sum"])
    correct_d.columns = ["dow_mean", "dow_sum"]

    df = pd.merge(df, correct_t, on=["testId"], how="left")
    df = pd.merge(df, correct_k, on=["KnowledgeTag"], how="left")
    df = pd.merge(df, correct_a, on=["assessmentItemID"], how="left")
    df = pd.merge(df, correct_p, on=["problem_number"], how="left")
    df = pd.merge(df, correct_h, on=["hour"], how="left")
    df = pd.merge(df, correct_d, on=["dow"], how="left")

    o_df = df[df["answerCode"] == 1]
    x_df = df[df["answerCode"] == 0]

    elp_k = df.groupby(["KnowledgeTag"])["elapsed"].agg("mean").reset_index()
    elp_k.columns = ["KnowledgeTag", "tag_elp"]
    elp_k_o = o_df.groupby(["KnowledgeTag"])["elapsed"].agg("mean").reset_index()
    elp_k_o.columns = ["KnowledgeTag", "tag_elp_o"]
    elp_k_x = x_df.groupby(["KnowledgeTag"])["elapsed"].agg("mean").reset_index()
    elp_k_x.columns = ["KnowledgeTag", "tag_elp_x"]

    df = pd.merge(df, elp_k, on=["KnowledgeTag"], how="left")
    df = pd.merge(df, elp_k_o, on=["KnowledgeTag"], how="left")
    df = pd.merge(df, elp_k_x, on=["KnowledgeTag"], how="left")

    ass_k = df.groupby(["assessmentItemID0"])["elapsed"].agg("mean").reset_index()
    ass_k.columns = ["assessmentItemID0", "ass_elp"]
    ass_k_o = o_df.groupby(["assessmentItemID0"])["elapsed"].agg("mean").reset_index()
    ass_k_o.columns = ["assessmentItemID0", "ass_elp_o"]
    ass_k_x = x_df.groupby(["assessmentItemID0"])["elapsed"].agg("mean").reset_index()
    ass_k_x.columns = ["assessmentItemID0", "ass_elp_x"]

    df = pd.merge(df, ass_k, on=["assessmentItemID0"], how="left")
    df = pd.merge(df, ass_k_o, on=["assessmentItemID0"], how="left")
    df = pd.merge(df, ass_k_x, on=["assessmentItemID0"], how="left")

    prb_k = df.groupby(["problem_number"])["elapsed"].agg("mean").reset_index()
    prb_k.columns = ["problem_number", "prb_elp"]
    prb_k_o = o_df.groupby(["problem_number"])["elapsed"].agg("mean").reset_index()
    prb_k_o.columns = ["problem_number", "prb_elp_o"]
    prb_k_x = x_df.groupby(["problem_number"])["elapsed"].agg("mean").reset_index()
    prb_k_x.columns = ["problem_number", "prb_elp_x"]

    df = pd.merge(df, prb_k, on=["problem_number"], how="left")
    df = pd.merge(df, prb_k_o, on=["problem_number"], how="left")
    df = pd.merge(df, prb_k_x, on=["problem_number"], how="left")

    df["user_correct_answer"] = (
        df.groupby("userID")["answerCode"]
        .transform(lambda x: x.cumsum().shift(1))
        .fillna(0)
    )
    df["user_total_answer"] = df.groupby("userID")["answerCode"].cumcount()
    df["user_acc"] = (df["user_correct_answer"] / df["user_total_answer"]).fillna(0)
    df["Grade_o"] = (
        df.groupby(["userID", "grade"])["answerCode"]
        .transform(lambda x: x.cumsum().shift(1))
        .fillna(0)
    )
    df["GradeCount"] = df.groupby(["userID", "grade"]).cumcount()
    df["GradeAcc"] = (df["Grade_o"] / df["GradeCount"]).fillna(0)
    df["GradeElp"] = (
        df.groupby(["userID", "grade"])["elapsed"]
        .transform(lambda x: x.cumsum())
        .fillna(0)
    )
    df["GradeMElp"] = df["GradeElp"] / [
        v if v != 0 else 1 for v in df["GradeCount"].values
    ]

    f = lambda x: len(set(x))
    test = df.groupby(["testId"]).agg({"problem_number": "max", "KnowledgeTag": f})
    test.reset_index(inplace=True)

    test.columns = ["testId", "problem_count", "tag_count"]

    df = pd.merge(df, test, on="testId", how="left")

    gdf = df[["userID", "testId", "problem_number", "grade", "Timestamp"]].sort_values(
        by=["userID", "grade", "Timestamp"]
    )
    gdf["buserID"] = gdf["userID"] != gdf["userID"].shift(1)
    gdf["bgrade"] = gdf["grade"] != gdf["grade"].shift(1)
    gdf["first"] = gdf[["buserID", "bgrade"]].any(axis=1).apply(lambda x: 1 - int(x))
    gdf["RepeatedTime"] = gdf["Timestamp"].diff().fillna(pd.Timedelta(seconds=0))
    gdf["RepeatedTime"] = (
        gdf["RepeatedTime"].apply(lambda x: x.total_seconds()) * gdf["first"]
    )
    df["RepeatedTime"] = gdf["RepeatedTime"].apply(lambda x: math.log(x + 1))

    df["prior_KnowledgeTag_frequency"] = df.groupby(
        ["userID", "KnowledgeTag"]
    ).cumcount()

    df["problem_position"] = df["problem_number"] / df["problem_count"]
    df["solve_order"] = df.groupby(["userID", "testId"]).cumcount()
    df["solve_order"] = (
        df["solve_order"]
        - df["problem_count"] * (df["solve_order"] > df["problem_count"]).apply(int)
        + 1
    )
    df["retest"] = (df["solve_order"] > df["problem_count"]).apply(int)
    T = df["solve_order"] != df["problem_number"]
    TT = T.shift(1)
    TT[0] = False
    df["solved_disorder"] = (TT.apply(lambda x: not x) & T).apply(int)

    #df["testId"] = df["testId"].apply(lambda x: int(x[1:4] + x[-3]))

    print("-" * 20, "Feature Engineering End", "-" * 20)
    print(f"Feature Engineering에 걸린 시간 : {time.time() - start_time}s")
    return df